In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hgnc = pd.read_csv('/Users/metinyazar/Desktop/drug_resistance_ppi/database_files/hgnc/hgnc_complete_set.txt',sep='\t')
columns_to_include = ['hgnc_id', 'symbol','prev_symbol','ensembl_gene_id','alias_symbol','entrez_id']
hgnc_dict = hgnc[columns_to_include].to_dict(orient='records')

In [ ]:
file = pd.read_csv('file.csv')

In [ ]:
# Build lookup dictionary
gene_lookup = {}

# Helper function to normalize symbols
def normalize_symbol(s):
    return s.strip().upper() if isinstance(s, str) else None

for entry in hgnc_dict:
    hgnc_id = entry.get('hgnc_id', np.nan)
    ensembl_id = entry.get('ensembl_gene_id', np.nan)
    entrez_id = str(entry.get('entrez_id', 'NA')) if pd.notnull(entry.get('entrez_id')) else 'NA'

    # Primary symbol
    symbol = normalize_symbol(entry.get('symbol'))
    if symbol:
        gene_lookup[symbol] = (hgnc_id, ensembl_id, entrez_id)

    # Previous symbols (only add if not already in gene_lookup)
    if pd.notnull(entry.get('prev_symbol')):
        prev_symbols = entry['prev_symbol'].split('|')
        for prev_symbol in prev_symbols:
            norm_prev = normalize_symbol(prev_symbol)
            if norm_prev and norm_prev not in gene_lookup:
                gene_lookup[norm_prev] = (hgnc_id, ensembl_id, entrez_id)

    # Alias symbols (only add if not already in gene_lookup)
    if pd.notnull(entry.get('alias_symbol')):
        alias_symbols = entry['alias_symbol'].split('|')
        for alias in alias_symbols:
            norm_alias = normalize_symbol(alias)
            if norm_alias and norm_alias not in gene_lookup:
                gene_lookup[norm_alias] = (hgnc_id, ensembl_id, entrez_id)

# Lookup function using normalized input
def lookup_gene_info(gene_symbol):
    key = normalize_symbol(gene_symbol)
    return gene_lookup.get(key, (np.nan, np.nan, 'NA'))

# Apply the lookup function to both Biomarker and Target1 columns
file['hgnc_id_1'], file['ensembl_gene_id_1'], file['entrez_id_1'] = zip(*file['lead_gene'].apply(lookup_gene_info))
file['hgnc_id_2'], file['ensembl_gene_id_2'], file['entrez_id_2'] = zip(*file['partner_gene'].apply(lookup_gene_info))
file